In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import matplotlib.pyplot as plt 

In [ ]:
#usuario = 'siempreread'
#contrasena = 'S13mpr3_Ch1l3'
#url_servidor = '209.151.148.250'
puerto = '3306'
esquema = 'EspoCRM2'
plugin_autenticacion = 'mysql_native_password'

usuario = 'care_consulta'
contrasena = 'CareAssistance!10'
url_servidor = '10.0.10.22'

# Conexión a la base de datos

c_conexion = 'mysql+mysqlconnector://{0}:{1}@{2}:{3}/{4}?auth_plugin={5}'
c_conexion = c_conexion.format(usuario, contrasena, url_servidor, puerto,
                               esquema, plugin_autenticacion)
motor_mysql_mariadb = create_engine(c_conexion)


#archivo de salida modificar segun archivo
#archivo = 'C:/Users/jmartinez/Desktop/casos nuevos/clientes/1/casos_faltantes_segun_archivo.xlsx'

In [ ]:
#ojo modificar año de la tabla  para ver bien los datos

sql = 'select *, month(created_at) as mes, year(created_at) as ano from contact where CHAR_LENGTH(rut_asegurado) > 6 AND CHAR_LENGTH(rut) > 6 '
#dataframe desde consulta SQL ######   and month(created_at) = 6
datos = pd.read_sql_query(sql, motor_mysql_mariadb)


datos.shape


In [ ]:
#tabla de emails
listado_email = 'SELECT t1.id,t3.lower as email FROM contact t1 left JOIN entity_email_address t2 ON t1.id = t2.entity_id left JOIN email_address t3 ON t2.email_address_id = t3.id '
emails = pd.read_sql_query(listado_email, motor_mysql_mariadb)
emails.shape

In [ ]:
#se juntan emails a tooodos los clientes 
data20 = pd.merge(left=datos,right=emails, how='left', left_on='id', right_on='id',copy=False)
data20.shape

In [ ]:
#limpieza tablas
data20.drop(['do_not_call','address_postal_code','marca_preexistencia','marca_programa_cronico','marca_enfermedad_alto_costo','marca_problemas_nutricionales','marca_trastornos_mentales','marca_momento_traumatico'], axis=1, inplace=True)
data20.drop(['url_empleador','marca_inicio_sintomas','marca_requirio_derivacion_emergencias','marca_requirio_internacion','marca_fecha_alta_seguimiento_virtual','middle_name'], axis=1, inplace=True)
data20.drop(['account_id','campaign_id','patologies','family_background','drug_history','hospital_and_surgeries_history','allergies','habits','habits_tobacco','habits_alcohol'],axis=1, inplace=True)
data20.drop(['habits_drugs','md_sedentarism','md_injuries_history','malnutrition','md_mental_illness','md_malnutrition','mes','ano'],axis=1, inplace=True)

#saca los espacios de todo el dataframe 
data20 = data20.replace(r"^ +| +$",r"",regex=True)

#cambia los valores en SEX
remplazahombremujer = {'Femenino': 'F', 'Masculino': 'M', '': 'O','Hombre':'M','Mujer':'F','M':'M','F':'F','O':'O','Ambos':'O','N':'O','Otro':'O','FEMENINO':'F','MASCULINO':'M'}
data20.replace({'sex': remplazahombremujer}, inplace=True)

#cambia los valores de relacion con titulares
relacion = {'ND':'O','ES':'CO','Titular':'AS','Cónyuge':'CO','Hijo/a':'HI','Otro':'O','Mamá':'M','Papá':'P','Nieto/a':'NI','TITULAR':'AS','CÓNYUGE':'CO','HIJO':'HI','HIJA':'HI','OTRO':'O','PAPA':'P','MAMA':'M','PAPÁ':'P','MAMÁ':'M','NIETO':'NI','NIETA':'NI','PADRE':'P','MADRE':'M','COTIZANTE':'AS','CONYUGE':'CO','Mama':'M','Pareja':'CO','CONYUGE/CONVIVIENTE':'CO','MA':'M','OT':'O','PA':'P','SO':'O'}
data20.replace({'relacion_con_asegurado': relacion}, inplace=True)

#saca acentos
acentos = {'Á':'A','É':'E','Í':'I','Ó':'O','Ú':'U','á':'a','é':'e','í':'i','ó':'o','ú':'u'}
data20.replace({'address_city': acentos}, inplace=True)
data20.replace({'address_state': acentos}, inplace=True)
data20.replace({'address_country': acentos}, inplace=True)

data20['relacion_con_asegurado'] = data20['relacion_con_asegurado'].str.upper()
data20['first_name'] = data20['first_name'].str.title()
data20['last_name'] = data20['last_name'].str.title()
data20['description'] = data20['description'].str.lower()
data20['address_street'] = data20['address_street'].str.title()
data20['address_city'] = data20['address_city'].str.title()
data20['address_state'] = data20['address_state'].str.title()
data20['address_country'] = data20['address_country'].str.title()
data20['campana_paciente'] = data20['campana_paciente'].str.upper()
data20['empleador'] = data20['empleador'].str.title()
data20['convenio'] = data20['convenio'].str.title()
data20['isapre'] = data20['isapre'].str.title()
data20['cargo'] = data20['cargo'].str.title()
data20['marca_estado_c19'] = data20['marca_estado_c19'].str.title()
data20['holding'] = data20['holding'].str.title()
data20['horariopreferente'].str.upper()
data20['division'] = data20['division'].str.title()
data20['ubicacion'] = data20['ubicacion'].str.title()
#data20['email'] = data20['email'].str.lower()


data20['address_city'] = data20['address_city'].str.strip('1234567890.#/}{)(¥|xX;,-_')
data20['address_state'] = data20['address_state'].str.strip('1234567890.#/}{)(¥|xX;,-_')
data20['address_street'] = data20['address_street'].str.strip(',.#/}{)(¥|xX;')
data20['address_country'] = data20['address_country'].str.strip('1234567890,.#/}{)(¥|xX;')

#sacar los puntos de los rut
data20['rut'] = data20['rut'].str.replace('.','')
data20['rut_asegurado'] = data20['rut_asegurado'].str.replace('.','')
data20['rut_asegurado'] = data20['rut_asegurado'].str.strip()
data20['rut'] = data20['rut'].str.strip()

data20['address_city'] =data20['address_city'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
data20['address_state'] =data20['address_state'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
data20['address_country'] =data20['address_country'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
#data20['address_city'] = data20['address_city'].str.rstrip('Region Metropolitana','Bahia','Sao Paulo','Buenos Aires','Capital Federal','Atlantico','Cordoba','Santa Fe','Rio Grande Do Sul','Rio De Janeiro','Santiago')
#data20['address_city'] = data20['address_city'].str.strip('1234567890.#/}{)(¥|xX;,-_')

data20['address_city'] = data20['address_city'].str.replace(', Region Metropolitana','')
data20['address_city'] = data20['address_city'].str.replace(', Bahia','')
data20['address_city'] = data20['address_city'].str.replace(', Sao Paulo','')
data20['address_city'] = data20['address_city'].str.replace(', Buenos Aires','')
data20['address_city'] = data20['address_city'].str.replace(', Capital Federal','')
data20['address_city'] = data20['address_city'].str.replace(', Atlantico','')
data20['address_city'] = data20['address_city'].str.replace(', Cordoba','')
data20['address_city'] = data20['address_city'].str.replace(', Santa Fe','')
data20['address_city'] = data20['address_city'].str.replace(', Rio Grande Do Sul','')
data20['address_city'] = data20['address_city'].str.replace(', Rio De Janeiro','')
data20['address_city'] = data20['address_city'].str.replace(', Santiago','')
data20['address_city'] = data20['address_city'].str.replace(', Santiago Metropolitan','')
data20['address_city'] = data20['address_city'].str.replace(', Santiago Metropolitana','')
data20['address_city'] = data20['address_city'].str.replace(', Morelos','')
data20['address_city'] = data20['address_city'].str.replace(', Concepcion','')
data20['address_city'] = data20['address_city'].str.replace(', San Jose','')
data20['address_city'] = data20['address_city'].str.replace(', San Pedro','')
data20['address_city'] = data20['address_city'].str.replace(', San Luis','')
data20['address_city'] = data20['address_city'].str.replace(', San Juan','')
data20['address_city'] = data20['address_city'].str.replace(', San Miguel','')
data20['address_city'] = data20['address_city'].str.replace(', San Martin','')
data20['address_city'] = data20['address_city'].str.replace(', San Salvador','')
data20['address_city'] = data20['address_city'].str.replace(', San Francisco','')
data20['address_city'] = data20['address_city'].str.replace(', San Bernardo','')
data20['address_city'] = data20['address_city'].str.replace(', San Carlos','')
data20['address_city'] = data20['address_city'].str.replace(', San Cristobal','')
data20['address_city'] = data20['address_city'].str.replace(', San Fernando','')
data20['address_city'] = data20['address_city'].str.replace(', Paraiba','')
data20['address_city'] = data20['address_city'].str.replace(', San Luis Potosi','')
data20['address_city'] = data20['address_city'].str.replace(', Concepcion','')
data20['address_city'] = data20['address_city'].str.replace(', Temuco','')
data20['address_city'] = data20['address_city'].str.replace('-Prov. De Bs.As.','')
data20['address_city'] = data20['address_city'].str.replace(' - Comuna 13','')
data20['address_city'] = data20['address_city'].str.replace(' - Buenos Aires','')
data20['address_city'] = data20['address_city'].str.replace(', Mendoza','')
data20['address_city'] = data20['address_city'].str.replace(', La Rioja','')
data20['address_city'] = data20['address_city'].str.replace(', Rio Negro','')
data20['address_city'] = data20['address_city'].str.replace(', Neuquen','')
data20['address_city'] = data20['address_city'].str.replace(', Tucuman','')
data20['address_city'] = data20['address_city'].str.replace(', Mexico State','')
data20['address_city'] = data20['address_city'].str.replace(', Distrito Federal','')
data20['address_city'] = data20['address_city'].str.replace(', Santa Catarina','')
data20['address_city'] = data20['address_city'].str.replace(', Santa Cruz','')
data20['address_city'] = data20['address_city'].str.replace(', Santa Fe','')
data20['address_city'] = data20['address_city'].str.replace(', Santa Elena','')
data20['address_city'] = data20['address_city'].str.replace(' - Capital Federal','')
data20['address_city'] = data20['address_city'].str.replace(' - Buenos Aires','')
data20['address_city'] = data20['address_city'].str.replace(' - Cordoba','')
data20['address_city'] = data20['address_city'].str.replace(' - Santa Fe','')
data20['address_city'] = data20['address_city'].str.replace(' - Rio Grande Do Sul','')
data20['address_city'] = data20['address_city'].str.replace(' - Rio De Janeiro','')
data20['address_city'] = data20['address_city'].str.replace(' - Santiago','')
data20['address_city'] = data20['address_city'].str.replace(', Entre Rios','')
data20['address_city'] = data20['address_city'].str.replace(', Chubut','')
data20['address_city'] = data20['address_city'].str.replace(', Chaco','')
data20['address_city'] = data20['address_city'].str.replace(', Catamarca','')
data20['address_city'] = data20['address_city'].str.replace(', Corrientes','')
data20['address_city'] = data20['address_city'].str.replace(', Tierra Del Fuego','')
data20['address_city'] = data20['address_city'].str.replace(', Entre Rios','')
data20['address_city'] = data20['address_city'].str.replace(', La Pampa','')
data20['address_city'] = data20['address_city'].str.replace(' - Isidro Casanova','')
data20['address_city'] = data20['address_city'].str.replace(' - San Miguel','')
data20['address_city'] = data20['address_city'].str.replace(' Area, Colombia','')
data20['address_city'] = data20['address_city'].str.replace(', Monte Grande','')
data20['address_city'] = data20['address_city'].str.replace(', Cundinamarca','')
data20['address_city'] = data20['address_city'].str.replace(', Canelones','')
data20['address_city'] = data20['address_city'].str.replace(', Bogota D.C','')
data20['address_city'] = data20['address_city'].str.replace(', La Plata','')
data20['address_city'] = data20['address_city'].str.replace(', Capital','') 
data20['address_city'] = data20['address_city'].str.replace(' Buenos Aires','')
data20['address_city'] = data20['address_city'].str.replace(', Valparaiso','')
data20['address_city'] = data20['address_city'].str.replace(', Chiapas','')
data20['address_city'] = data20['address_city'].str.replace(', Veracruz','')
data20['address_city'] = data20['address_city'].str.replace(', Minas Gerais','')
data20['address_city'] = data20['address_city'].str.replace(', Espirito Santo','')
data20['address_city'] = data20['address_city'].str.replace(', Mato Grosso Do Sul','')
data20['address_city'] = data20['address_city'].str.replace(', Mato Grosso','')
data20['address_city'] = data20['address_city'].str.replace(', Minas Gerais','')
data20['address_city'] = data20['address_city'].str.replace(', Hidalgo','')
data20['address_city'] = data20['address_city'].str.replace(' - Monte Grande','')
data20['address_city'] = data20['address_city'].str.replace(', Pernambuco','')
data20['address_city'] = data20['address_city'].str.replace(', Dependencias Federales','')
data20['address_city'] = data20['address_city'].str.replace(', Tigre','')
data20['address_city'] = data20['address_city'].str.replace(', Lanus','')
data20['address_city'] = data20['address_city'].str.replace(', San Luis','')
data20['address_city'] = data20['address_city'].str.replace(' , Solano','')
data20['address_city'] = data20['address_city'].str.replace(' / Juana Koslay','')  
data20['address_city'] = data20['address_city'].str.replace(' - Vicente Lopez','')
data20['address_city'] = data20['address_city'].str.replace(' - San Isidro','')
data20['address_city'] = data20['address_city'].str.replace(' - Mendoza','')
data20['address_city'] = data20['address_city'].str.replace(', Vicente Lopez','')
data20['address_city'] = data20['address_city'].str.replace(' - Vicente Lopez','')
data20['address_city'] = data20['address_city'].str.replace(', Vicente Lopez','')
data20['address_city'] = data20['address_city'].str.replace(', San Isidro','')
data20['address_city'] = data20['address_city'].str.replace('. Cordoba','')
data20['address_city'] = data20['address_city'].str.replace(' - Avellaneda','')
data20['address_city'] = data20['address_city'].str.replace(' ( La Matanza )','')
data20['address_city'] = data20['address_city'].str.replace(' - La Tablada','')
data20['address_city'] = data20['address_city'].str.replace(', Vicente Lopez','')
data20['address_city'] = data20['address_city'].str.replace(' - Gregorio De Laferrere','')
data20['address_city'] = data20['address_city'].str.replace(', Metropolitana De Santiago','')
data20['address_city'] = data20['address_city'].str.replace(', Sinaloa','')
data20['address_city'] = data20['address_city'].str.replace(', Nuevo Leon','')
data20['address_city'] = data20['address_city'].str.replace(', Campeche','')
data20['address_city'] = data20['address_city'].str.replace(', Jalisco','')
data20['address_city'] = data20['address_city'].str.replace(', Tamaulipas','')
data20['address_city'] = data20['address_city'].str.replace(', Yucatan','')
data20['address_city'] = data20['address_city'].str.replace(', Quintana Roo','')
data20['address_city'] = data20['address_city'].str.replace(', Sonora','')
data20['address_city'] = data20['address_city'].str.replace(', Queretaro','')
data20['address_city'] = data20['address_city'].str.replace(', Sonora','')
data20['address_city'] = data20['address_city'].str.replace(', Goias','')
data20['address_city'] = data20['address_city'].str.replace(', Parana','')
data20['address_city'] = data20['address_city'].str.replace(', Amazonas','')
data20['address_city'] = data20['address_city'].str.replace(', Uruguay','')
data20['address_city'] = data20['address_city'].str.replace(', Miranda','')
data20['address_city'] = data20['address_city'].str.replace(', Caracas','')
data20['address_city'] = data20['address_city'].str.replace(', Sucre','')
data20['address_city'] = data20['address_city'].str.replace(', Antioquia','')
data20['address_city'] = data20['address_city'].str.replace(' - Maschwitz','')
data20['address_city'] = data20['address_city'].str.replace('- Temperley','')
data20['address_city'] = data20['address_city'].str.replace(' (Parque Avellaneda)','')
data20['address_city'] = data20['address_city'].str.replace(', Coahuila','')
data20['address_city'] = data20['address_city'].str.replace(', Baja California','')
data20['address_city'] = data20['address_city'].str.replace(', Alagoas','')
data20['address_city'] = data20['address_city'].str.replace(' - Sp','')
data20['address_city'] = data20['address_city'].str.replace(' - 13 - Region Metropolitana De Santiago - Chile','')
data20['address_city'] = data20['address_city'].str.replace(', Octava Region','')
data20['address_city'] = data20['address_city'].str.replace(' - 08 - Region Del Biobio - Chile','')
data20['address_city'] = data20['address_city'].str.replace(' - 15 - Arica Y Parinacota - Chile','')
data20['address_city'] = data20['address_city'].str.replace(' - 05 - Valparaiso - Chile','')
data20['address_city'] = data20['address_city'].str.replace(' - 03 - Atacama - Chile','')
data20['address_city'] = data20['address_city'].str.replace(' - 10 - Region De Los Lagos - Chile','')
data20['address_city'] = data20['address_city'].str.replace(' - 02 - Antofagasta - Chile','')
data20['address_city'] = data20['address_city'].str.replace(' - 01 - Tarapaca - Chile','')
data20['address_city'] = data20['address_city'].str.replace(' - 07 - Region Del Maule - Chile','')
data20['address_city'] = data20['address_city'].str.replace(' - 16 - Region De Nuble - Chile','')
data20['address_city'] = data20['address_city'].str.replace(' - 14 - Region De Los Rios - Chile','')
data20['address_city'] = data20['address_city'].str.replace(' - 04 - Coquimbo - Chile','')
data20['address_city'] = data20['address_city'].str.replace(' - 06 - Region Del Libertador General Bernardo O\'higgins - Chile','')
data20['address_city'] = data20['address_city'].str.replace(' - 03 - Atacama - Chile','')
data20['address_city'] = data20['address_city'].str.replace(' - 09 - Region De Aysen Del General Carlos Ibanez Del Campo - Chile','')
data20['address_city'] = data20['address_city'].str.replace(' - 12 - Region De Magallanes Y De La Antartica Chilena - Chile','')
data20['address_city'] = data20['address_city'].str.replace(' - 15 - Arica Y Parinacota - Chile','')
data20['address_city'] = data20['address_city'].str.replace(' - 07 - Region Del Maule - Chile','')
data20['address_city'] = data20['address_city'].str.replace(' - 11 - Region Aisen Del Gral. Carlos Ibanez Del Camp - Chile','')
data20['address_city'] = data20['address_city'].str.replace(' 6 Ta Region','')
data20['address_city'] = data20['address_city'].str.replace(', Aysen','')



ciudades = {
'[No Especificado]':'',
'Region Metropolitana':'Santiago',
'No Informado':'',
'Metropolitana':'Santiago',
', Bahia':'',', Sao Paulo':'',
'Nuble':'Ñuble',
', Buenos Aires':'',
'Capital Federal':'Buenos Aires',
'Atlantico':'',
', Cordoba':'',
', Santa Fe':'',
', Rio Grande Do Sul':'',
', Rio De Janeiro':'',
', Santiago':'',
', Cordoba':'',
', Morelos':'',
', Paraiba':'',
', Santa Catarina':'',
'- Alto Macul':'',
'Metropolitana De Santiago':'Santiago',
', Temuco':'',
', Concepcion':'',
'Vina Del Mar':'Viña Del Mar',
'Nunoa':'Ñuñoa',
'Stgo':'Santiago',
'Santiago Centro':'Santiago',
'Vicuna':'Vicuña',
'Region De La Araucania':'Araucanía',
'Provincia Cordillera':'Cordillera',
'Provincia De Copiapo':'Copiapó',
'Buenos Aires Capital':'Buenos Aires',
'S\C':''
}
data20.replace({'address_city': ciudades}, inplace=True)
data20.replace({'address_state': ciudades}, inplace=True)
data20.replace({'address_country': ciudades}, inplace=True)

#saca los espacios de todo el dataframe
data20 = data20.replace(r"^ +| +$",r"",regex=True)
#data20['contar'] = data20['LLave'].str.contains(data20['LLave']).value_counts()[True]
#crea llave unica



In [ ]:
def cambiarSa(x):      
    #subcadena = 'Sa' #la subcadena que queremos
    #if subcadena in x and x.index(subcadena) != 0:
    
    if x[len(x)-2:] == 'Sa':
        x = x.replace('Sa','S.A.')
    if x[len(x)-2:] == 'S.':
        x = x.replace('S.','S.A.')
    if x[len(x)-3:] == 'S A':
        x = x.replace('S A','S.A.')
    if x[len(x)-3:] == 'SA.':
        x = x.replace('SA.','S.A.')
    if x[len(x)-3:] == 'S.A':
        x = x.replace('S.A','S.A.') 
    if x[len(x)-4:] == 'S. A': 
        x = x.replace('S. A','S.A.') 
    return x

def cambiarIn(y):
    
    if y[0:4] == 'Soc ':
        y = y.replace('Soc ','Soc. ')
    if y[0:5] == 'Serv ':
        y = y.replace('Serv ','Serv. ')
    if y[0:5] == 'Asoc ':
        y = y.replace('Asoc ','Asoc. ')
    return y

In [ ]:
#ingresar data20 segun campaña


data20.empleador = data20['empleador'].astype(str)

#saca acentos

#saca espacios

data20['empleador'] = data20['empleador'].str.strip()
data20['empleador'] = data20['empleador'].str.strip('.')
data20['holding'] = data20['holding'].str.strip('0123456789-_.,')
data20['holding'] = data20['holding'].str.strip()
data20['holding'] = data20['holding'].str.strip('.')



#los deja en mayuscula
data20['empleador'] = data20['empleador'].str.upper()
#los deja con la primera letra con mayuscula y el resto minuscula en todas las palabras
#data20['empleador'] = data20['empleador'].str.replace('S A','S.A.')
#data20['empleador'] = data20['empleador'].str.replace('SA.','S.A.')
#data20['empleador'] = data20['empleador'].str.replace('S.A','S.A.')
data20['empleador'] = data20['empleador'].str.replace('LTDA','Ltda.')
data20['empleador'] = data20['empleador'].str.replace('SAEZA.','SAESA.')
#data20['empleador'] = data20['empleador'].str.replace('s.A.','sa')

data20['empleador'] = data20['empleador'].str.replace('Á','A')
data20['empleador'] = data20['empleador'].str.replace('É','E')
data20['empleador'] = data20['empleador'].str.replace('Í','I')
data20['empleador'] = data20['empleador'].str.replace('Ó','O')
data20['empleador'] = data20['empleador'].str.replace('Ú','U')


data20['empleador'] = data20['empleador'].str.title()
data20['empleador'] = data20['empleador'].str.replace('Spa','Spa.')
data20['empleador'] = data20['empleador'].str.replace('Ltda..','Ltda.')
data20['empleador'] = data20['empleador'].str.replace('Limitada','Ltda.')
data20['empleador'] = data20['empleador'].str.replace('Siempre Chile','Care Assistance')
data20['empleador'] = data20['empleador'].str.replace('Colaboradores Empresas','Care Assistance')
data20['empleador'] = data20['empleador'].str.replace('Clalemana','Clinica Alemana')
data20['empleador'] = data20['empleador'].str.replace('Bancodechile','Banco de Chile')
data20['empleador'] = data20['empleador'].str.replace('Bancochile','Banco de Chile')
data20['empleador'] = data20['empleador'].str.replace('Conchaytoro','Concha y Toro')
data20['empleador'] = data20['empleador'].str.replace('Prueba Empresas','Testing De Ejecutivos')
data20['empleador'] = data20['empleador'].str.replace('Serv_Vicio','Servicio')
data20['empleador'] = data20['empleador'].str.replace('Socidad','Sociedad')
data20['empleador'] = data20['empleador'].str.replace('Serv.Bienestar','Serv. Bienestar')
data20['empleador'] = data20['empleador'].str.replace('S.P.A','Spa.')
data20['empleador'] = data20['empleador'].str.replace('10 Uf Asegurados En Evaluacion','Asegurados En Evaluacion')

data20['empleador'] = data20['empleador'].str.replace('Lmtda','Ltda.')

data20['empleador'] = data20['empleador'].apply(lambda x: cambiarSa(x))
data20['empleador'] = data20['empleador'].apply(lambda y: cambiarIn(y))



data20.convenio.fillna(data20.campana_paciente + 'convenio', inplace=True)
data20['convenio'] = data20['convenio'].str.lower()
convenios = {
     'metlifeconvenio':'Orienta',
     '3spconvenio':'3SP',
     'ambar mxconvenio':'Ambar',
     'asismedconvenio':'Asismed',
     'bciconvenio':'BCI',
     'burgos - consorcioconvenio':'Burgos - Consorcio'

}
data20.replace({'convenio': convenios}, inplace=True)
data20['convenio'] = data20['convenio'].str.lower()
data20['campana_paciente'] = data20['campana_paciente'].str.title() 
data20['holding'] = data20['holding'].str.title()

data20.empleador.fillna(data20.campana_paciente + ' empresa', inplace=True)
data20.holding.fillna('Holding ' + data20.campana_paciente, inplace=True)

#llave del sistema
data20['LLave'] = data20['rut'].astype(str) + data20['rut_asegurado'].astype(str) + data20['campana_paciente'].astype(str) + data20['empleador'].astype(str)

In [ ]:
data20['empleador'].head

In [28]:
faltantes_mr1 =  'C:/Users/jmartinez/Desktop/faltantes/1.xlsx'
faltantes_mr2 =  'C:/Users/jmartinez/Desktop/faltantes/2.xlsx'
faltantes_mr3 =  'C:/Users/jmartinez/Desktop/faltantes/3.xlsx'

faltantes = pd.read_excel(faltantes_mr1)
faltantes2 = pd.read_excel(faltantes_mr2)
faltantes3 = pd.read_excel(faltantes_mr3)

In [29]:
faltantes['rut'] = faltantes['rut'].str.strip()
faltantes['rut'] = faltantes['rut'].str.lstrip()
faltantes['rut'] = faltantes['rut'].str.rstrip()
faltantes['rut'] = faltantes['rut'].str.lower()
faltantes['rut'] = faltantes['rut'].str.strip()
faltantes['rut'] = faltantes['rut'].str.replace(' ','')
data20['rut'] = data20['rut'].str.strip()
data20['rut'] = data20['rut'].str.lstrip()
data20['rut'] = data20['rut'].str.rstrip()
data20['rut'] = data20['rut'].str.lower()
data20['rut'] = data20['rut'].str.strip()
data20['rut'] = data20['rut'].str.replace(' ','')
faltantes_df = pd.merge(left=faltantes,right=data20,how='left',left_on='rut',right_on='rut')
llave1 = faltantes_df.value_counts(['LLave'],sort=True).to_frame()
faltantes_df_fin = pd.merge(left=faltantes_df,right=llave1, how='left', left_on='LLave', right_on='LLave')
faltantes_df_fin = faltantes_df_fin.drop_duplicates(subset = ['LLave'], keep='last')
faltantes_df_fin.to_excel('C:/Users/jmartinez/Desktop/faltantes/11.xlsx',index=False)

faltantes_df_fin.shape

(47, 36)

In [30]:
faltantes_df_fin.head()

,rut,id,deleted,salutation_name,first_name,last_name,description,address_street,address_city,address_state,...,created_by_id,modified_by_id,assigned_user_id,holding,horariopreferente,division,ubicacion,email,LLave,0
1,12330821-2,60df9d012f84ce0f5,0.0,None,Contreras,Contreras Juana,None,Sector Cuyampalihue Quelen Quelen,,,...,system,system,None,Holding Metlife,,None,None,contrerasjuany77@gmail.com,12330821-212330821-2MetlifeRendic Hnos. S.A.,1.0
2,12330821-2,60f919ed8a6e0e6fd,0.0,None,Juana,Contreras Contreras,None,None,None,None,...,system,None,None,Holding Metlife,,None,None,None,12330821-212330821-2MetlifeLa Araucana C.C.A.F,1.0
3,12618246-5,60df9d060741b3005,0.0,None,Rojas,Escobar Hector Antonio,None,Osorno 1115,Calama,,...,system,system,None,Holding Metlife,,None,None,rojasescobar09@gmail.com,12618246-512618246-5MetlifeRendic Hnos. S.A.,1.0
4,12618246-5,60f919d3d78ae470f,0.0,None,Hector,Antonio Rojas Escobar,None,None,None,None,...,system,None,None,Holding Metlife,,None,None,None,12618246-512618246-5MetlifeLa Araucana C.C.A.F,1.0
5,12685856-6,627ac79c70f43f2f9,0.0,None,Sandra,Salamanca Castillo,None,"Pje Los Olivos 5752, Peñalolén",Santiago,,...,system,system,None,Holding Metlife,,None,None,ssalamancac@gmail.com,12685856-612685856-6MetlifeSacyr Chile Servicios Corporativos S.A.,1.0


In [31]:
faltantes2['rut'] = faltantes2['rut'].str.strip()
faltantes2['rut'] = faltantes2['rut'].str.lstrip()
faltantes2['rut'] = faltantes2['rut'].str.rstrip()
faltantes2['rut'] = faltantes2['rut'].str.lower()
faltantes2['rut'] = faltantes2['rut'].str.strip()
faltantes2['rut'] = faltantes2['rut'].str.replace(' ','')
data20['rut'] = data20['rut'].str.strip()
data20['rut'] = data20['rut'].str.lstrip()
data20['rut'] = data20['rut'].str.rstrip()
data20['rut'] = data20['rut'].str.lower()
data20['rut'] = data20['rut'].str.strip()
data20['rut'] = data20['rut'].str.replace(' ','')
faltantes_df2 = pd.merge(left=faltantes2,right=data20,how='left',left_on='rut',right_on='rut')
llave3 = faltantes_df2.value_counts(['LLave'],sort=True).to_frame()
faltantes_df_fin2 = pd.merge(left=faltantes_df2,right=llave3, how='left', left_on='LLave', right_on='LLave')
faltantes_df_fin2 = faltantes_df_fin2.drop_duplicates(subset = ['LLave'], keep='last')
faltantes_df_fin2.to_excel('C:/Users/jmartinez/Desktop/faltantes/22.xlsx',index=False)

faltantes_df_fin2.shape

(50, 36)

In [32]:
faltantes_df_fin2.head()

,rut,id,deleted,salutation_name,first_name,last_name,description,address_street,address_city,address_state,...,created_by_id,modified_by_id,assigned_user_id,holding,horariopreferente,division,ubicacion,email,LLave,0
0,20484077-6,5e4f7fb2b9d92e0a8,0,None,Josefa,Arratia Zuñiga,None,None,Quillota,Quillota,...,system,system,None,Holding Vida Camara,,None,None,marratia@agropuelma.cl,20484077-611827381-8Vida CamaraAgropuelma Ltda.,1
1,20484077-6,616e9f4ea4075ef85,0,None,Josefa,Arratia Zuñiga,None,"Bien Común Especial 9 Lote B, Lliu Liu",Limache,,...,system,system,None,Soquimich Comercial S.A,,None,None,joo.mirentxu@gmail.com,20484077-611827381-8Vida SecuritySoquimich Comercial S.A.,1
2,20528730-2,62b31e2dc9ca0344e,0,None,Javiera Constan,Inzunza Orellana,None,Avda Las Terrazas Nº 1335,Concepcion,None,...,system,system,None,Holding Metlife,,None,None,inzunzaorellana@hotmail.com,20528730-212984954-1MetlifeTecnofarma S.A.,1
3,22420492-2,6272c6b05093200b4,0,None,Valentina,Canales Salinas,None,"Las Parcelas 133, Coquimbo",,,...,system,5d922539c47ab4aba,None,Holding Metlife,,None,None,mazielsalinas@gmail.com,22420492-218076989-7MetlifeEnaex,1
4,22810080-3,5fa2e7ac373c0de18,0,None,Bravo,Galvez Sofia,None,Villamaria Requinoa,,,...,system,system,None,Holding Metlife,,None,None,23mochi13@gmail.com,22810080-315520781-7MetlifeAgrosuper,1


In [34]:
faltantes3['rut'] = faltantes3['rut'].str.strip()
faltantes3['rut'] = faltantes3['rut'].str.lstrip()
faltantes3['rut'] = faltantes3['rut'].str.rstrip()
faltantes3['rut'] = faltantes3['rut'].str.lower()
faltantes3['rut'] = faltantes3['rut'].str.strip()
faltantes3['rut'] = faltantes3['rut'].str.replace(' ','')
data20['rut'] = data20['rut'].str.strip()
data20['rut'] = data20['rut'].str.lstrip()
data20['rut'] = data20['rut'].str.rstrip()
data20['rut'] = data20['rut'].str.lower()
data20['rut'] = data20['rut'].str.strip()
data20['rut'] = data20['rut'].str.replace(' ','')
faltantes_df3 = pd.merge(left=faltantes3,right=data20,how='left',left_on='rut',right_on='rut')
llave2 = faltantes_df3.value_counts(['LLave'],sort=True).to_frame()
faltantes_df_fin3 = pd.merge(left=faltantes_df3,right=llave2, how='left', left_on='LLave', right_on='LLave')
faltantes_df_fin3 = faltantes_df_fin3.drop_duplicates(subset = ['LLave'], keep='last')
faltantes_df_fin3.to_excel('C:/Users/jmartinez/Desktop/faltantes/33.xlsx',index=False)

faltantes_df_fin3.shape

(66, 36)